# Homework 1: Filtering spam messages

**Ing 5 BDA grp1:**

Antoniadis Pablo

LLobregat Thomas

Ye Sebastien


# 0. Import the dataset from message.txt


*   import numpy as np
*   import pandas as pd
*   import matplotlib.pyplot as plt


In [17]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = "/content/drive/My Drive/Ing5/Machine learning/homeworks/HM 1: Filtering spam messages/"

os.chdir(path)

with open(path+'messages.txt', 'r') as db:
  data = db.readlines()
 

Mounted at /content/drive


In [18]:
 # little bit of data cleaning

 #remove '\n' at the end of each email  
data_length = len(data)
for i in range(data_length):
  data[i] = data[i][:-2]

# 1. Divide the data in two groups: training and test examples.

We decided to divide the data in two groups the training_set that has 70% of the data and test_set that has 30% 

In [19]:
data_length = len(data) #5000

training_set = [] # 70% = 3500
test_set = [] #30% = 1500

for i in range(data_length):
  if(i<data_length*0.7):
    training_set.append(data[i])
  else:
    test_set.append(data[i])
  
print("training_set length : ",len(training_set),"\ntest_set length : ",len(test_set))

training_set length :  3500 
test_set length :  1500


# 2. Parse both the training and test examples to generate both the spam and ham data sets.

In [20]:
X_train = []
y_train = []



for i in range(len(training_set)):
  y , X = training_set[i].split('\t')
  y_train.append(y)
  X_train.append(X)




In [21]:
X_test = []
y_test = []

for i in range(len(test_set)):
  y , X = test_set[i].split('\t')
  y_test.append(y)
  X_test.append(X)

# 3. Generate a dictionary from the training data.


In [22]:
training_data_frame = pd.DataFrame(list(zip(y_train, X_train)), 
               columns =['value => y_train', 'mail => X_train']) 

training_data_frame.head()

,value => y_train,mail => X_train
0,ham,Yup i've finished c ü there..
1,ham,Remember to ask alex about his pizz
2,ham,No da..today also i forgot.
3,ham,Ola would get back to you maybe not today but ...
4,ham,Fwiw the reason I'm only around when it's time...


In [23]:
test_data_frame = pd.DataFrame(list(zip(y_test, X_test)), 
               columns =['value => y_test', 'mail => X_test']) 


test_data_frame.head()



,value => y_test,mail => X_test
0,ham,Ok. She'll be ok. I gues
1,ham,aathi..where are you dear.
2,ham,Any pain on urination any thing else
3,ham,7 at esplanade.. Do ü mind giving me a lift co...
4,ham,I wnt to buy a BMW car urgently..its vry urgen...


# 4. Extract features from both the training data and test data.

## Getting all unique words first

In [24]:
words_set = set()

for mail in training_data_frame["mail => X_train"]:
  words = mail.split()
  for word in words:
    words_set.add(word)

print(words_set)
print(len(words_set))

{'She', 'bat:', 'mumHas', 'alone', 'book', 'thin', 'back', 'minecraft', 'malaria.', 'Stop', 'Actually', 'Cust', 'Goodnight', 'Hell', 'bringing', 'sum1', 'regards', 'credits!', 'Missing*', "You'll", 'foot', '£s', 'Claim', 'little.', '9A', "'Comfort'", 'IG11', 'ass!', '***********', 'cum..', 'doinat', '08452810073', 'girlfrnd...', '8007', 'Ha!', 'regard', 'relation..', 'mths?', 'King', 'try!', 'making', 'campus.', 'leav', 'when...', 'day,', 'social', 'tv..he', 'part', 'hostel.', 'Saw', 'trip...', 'refused?', 'things', 'Secret', 'greeting..', "wudn't", '7.', 'callon', 'Lets', 'FUCKING', 'battery', 'skint', 'Hoping', 'blonde,', 'sooner', 'SUM', "\x93It's", 'thanx...', 'movie..wat', 'pig', 'chicken', 'scraped', 'Toll', 'goods', 'GENDER', 'Old', "child's", '4.Cook:', 'Love', ':)going', '*snuggles', 'HG/Suite342/2lands', 'Props', 'you.:)k:)where', 'FROM', '47', 'you‘ll', 'JADE', 'POLY', 'LookAtMe!,', 'KAVALAN', 'ligh', 'Islands', 'owed', 'Eek', 'WOULD', 'come..til', '09061790125', 'lot..', 'f

In [25]:
for word in words_set:
  # count occurence for word in each row (email)
  occurences = []

  for mail in training_data_frame["mail => X_train"]:
      occurences.append(mail.count(word))

  training_data_frame.insert(0, word, occurences, False) 

In [26]:
training_data_frame.head()

,system,saturday,msg@£1.50rcv,released,text?,shouted,good!,TsCs087147403231Winawk!Age16,"jewelry,",luv!,slave!,discuss,Prize.,Jay,FOREVER,http://alto18.co.uk/wave/wave.asp?o=4434,5000,Handset?,already?,midnigh,office,funny,reboot,underdtand,area!,pix,"too,",inf,along,piece,5WQ,www.SMS.ac/u/bootydelious,madam,WHILE,C's.,sofa,days,3UZ.,jenny,doing?how,...,mths?,relation..,regard,Ha!,8007,girlfrnd...,08452810073,doinat,cum..,***********,ass!,IG11,'Comfort',9A,little.,Claim,£s,foot,You'll,Missing*,credits!,regards,sum1,bringing,Hell,Goodnight,Cust,Actually,Stop,malaria.,minecraft,back,thin,book,alone,mumHas,bat:,She,value => y_train,mail => X_train
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ham,Yup i've finished c ü there..
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ham,Remember to ask alex about his pizz
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ham,No da..today also i forgot.
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,ham,Ola would get back to you maybe not today but ...
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ham,Fwiw the reason I'm only around when it's time...


In [27]:
for word in words_set:
  # count occurence for word in each row
  occurences = []

  for mail in test_data_frame["mail => X_test"]:
      occurences.append(mail.count(word))

  test_data_frame.insert(0, word, occurences, False) 

In [28]:
test_data_frame.head()

,system,saturday,msg@£1.50rcv,released,text?,shouted,good!,TsCs087147403231Winawk!Age16,"jewelry,",luv!,slave!,discuss,Prize.,Jay,FOREVER,http://alto18.co.uk/wave/wave.asp?o=4434,5000,Handset?,already?,midnigh,office,funny,reboot,underdtand,area!,pix,"too,",inf,along,piece,5WQ,www.SMS.ac/u/bootydelious,madam,WHILE,C's.,sofa,days,3UZ.,jenny,doing?how,...,mths?,relation..,regard,Ha!,8007,girlfrnd...,08452810073,doinat,cum..,***********,ass!,IG11,'Comfort',9A,little.,Claim,£s,foot,You'll,Missing*,credits!,regards,sum1,bringing,Hell,Goodnight,Cust,Actually,Stop,malaria.,minecraft,back,thin,book,alone,mumHas,bat:,She,value => y_test,mail => X_test
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,ham,Ok. She'll be ok. I gues
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ham,aathi..where are you dear.
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ham,Any pain on urination any thing else
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ham,7 at esplanade.. Do ü mind giving me a lift co...
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ham,I wnt to buy a BMW car urgently..its vry urgen...


In [29]:
training_data_frame.drop(columns=["mail => X_train"], inplace=True)
training_data_frame['value => y_train'].replace(["ham", "spam"], [0, 1], inplace=True) #replacing ham by 0 and spam by 1

training_data_frame.describe()

,system,saturday,msg@£1.50rcv,released,text?,shouted,good!,TsCs087147403231Winawk!Age16,"jewelry,",luv!,slave!,discuss,Prize.,Jay,FOREVER,http://alto18.co.uk/wave/wave.asp?o=4434,5000,Handset?,already?,midnigh,office,funny,reboot,underdtand,area!,pix,"too,",inf,along,piece,5WQ,www.SMS.ac/u/bootydelious,madam,WHILE,C's.,sofa,days,3UZ.,jenny,doing?how,...,King,mths?,relation..,regard,Ha!,8007,girlfrnd...,08452810073,doinat,cum..,***********,ass!,IG11,'Comfort',9A,little.,Claim,£s,foot,You'll,Missing*,credits!,regards,sum1,bringing,Hell,Goodnight,Cust,Actually,Stop,malaria.,minecraft,back,thin,book,alone,mumHas,bat:,She,value => y_train
count,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,...,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000
mean,0.001143,0.002000,0.000286,0.000857,0.000286,0.000286,0.000857,0.000571,0.000286,0.001143,0.000286,0.001429,0.000286,0.000857,0.000286,0.000286,0.006857,0.000571,0.000571,0.000571,0.005714,0.000571,0.000857,0.000286,0.000286,0.001143,0.000857,0.010286,0.000857,0.000286,0.002286,0.000286,0.000571,0.000286,0.000286,0.002000,0.012000,0.000571,0.000286,0.000571,...,0.000857,0.000571,0.000286,0.001143,0.000286,0.003429,0.000571,0.000571,0.000286,0.000286,0.000571,0.000286,0.000286,0.000286,0.000571,0.000286,0.006857,0.000286,0.002857,0.001143,0.000286,0.000286,0.000571,0.000857,0.000571,0.007714,0.000571,0.004000,0.001714,0.004000,0.001143,0.000286,0.027429,0.087429,0.008000,0.002857,0.000286,0.000286,0.012857,0.133143
std,0.033792,0.044683,0.016903,0.029269,0.016903,0.016903,0.029269,0.023901,0.016903,0.033792,0.016903,0.037775,0.016903,0.029269,0.016903,0.016903,0.082535,0.023901,0.023901,0.023901,0.075387,0.023901,0.029269,0.016903,0.016903,0.033792,0.029269,0.103704,0.029269,0.016903,0.047761,0.016903,0.023901,0.016903,0.016903,0.044683,0.111494,0.023901,0.016903,0.023901,...,0.037792,0.023901,0.016903,0.033792,0.016903,0.071642,0.023901,0.023901,0.016903,0.016903,0.023901,0.016903,0.016903,0.016903,0.023901,0.016903,0.082535,0.016903,0.067562,0.033792,0.016903,0.016903,0.023901,0.029269,0.023901,0.090711,0.023901,0.063128,0.041374,0.063128,0.047802,0.016903,0.170206,0.314118,0.089097,0.053383,0.016903,0.016903,0.137779,0.339778
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [30]:
test_data_frame.drop(columns=["mail => X_test"], inplace=True)
test_data_frame['value => y_test'].replace(["ham", "spam"], [0, 1], inplace=True) #replacing ham by 0 and spam by 1

test_data_frame.describe()

,system,saturday,msg@£1.50rcv,released,text?,shouted,good!,TsCs087147403231Winawk!Age16,"jewelry,",luv!,slave!,discuss,Prize.,Jay,FOREVER,http://alto18.co.uk/wave/wave.asp?o=4434,5000,Handset?,already?,midnigh,office,funny,reboot,underdtand,area!,pix,"too,",inf,along,piece,5WQ,www.SMS.ac/u/bootydelious,madam,WHILE,C's.,sofa,days,3UZ.,jenny,doing?how,...,King,mths?,relation..,regard,Ha!,8007,girlfrnd...,08452810073,doinat,cum..,***********,ass!,IG11,'Comfort',9A,little.,Claim,£s,foot,You'll,Missing*,credits!,regards,sum1,bringing,Hell,Goodnight,Cust,Actually,Stop,malaria.,minecraft,back,thin,book,alone,mumHas,bat:,She,value => y_test
count,1500.000000,1500.000000,1500.0,1500.0,1500.0,1500.000000,1500.0,1500.000000,1500.0,1500.000000,1500.0,1500.0,1500.0,1500.000000,1500.0,1500.0,1500.000000,1500.0,1500.000000,1500.0,1500.000000,1500.000000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.000000,1500.000000,1500.0,1500.0,...,1500.000000,1500.0,1500.0,1500.000000,1500.0,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.0,1500.000000,1500.0,1500.0,1500.000000,1500.0,1500.000000,1500.0,1500.0,1500.0,1500.000000,1500.0,1500.000000,1500.0,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.0,1500.000000,1500.000000
mean,0.001333,0.001333,0.0,0.0,0.0,0.000667,0.0,0.000667,0.0,0.001333,0.0,0.0,0.0,0.002000,0.0,0.0,0.010000,0.0,0.001333,0.0,0.004667,0.000667,0.0,0.0,0.0,0.0,0.0,0.007333,0.000667,0.002000,0.002667,0.0,0.0,0.0,0.0,0.0,0.013333,0.000667,0.0,0.0,...,0.001333,0.0,0.0,0.002000,0.0,0.003333,0.000667,0.000667,0.0,0.0,0.0,0.000667,0.0,0.0,0.000667,0.0,0.006000,0.0,0.0,0.0,0.000667,0.0,0.001333,0.0,0.001333,0.007333,0.001333,0.002667,0.002000,0.002000,0.0,0.0,0.018000,0.096667,0.008667,0.002667,0.0,0.0,0.007333,0.137333
std,0.036503,0.036503,0.0,0.0,0.0,0.025820,0.0,0.025820,0.0,0.036503,0.0,0.0,0.0,0.044692,0.0,0.0,0.112139,0.0,0.036503,0.0,0.068176,0.025820,0.0,0.0,0.0,0.0,0.0,0.085349,0.025820,0.044692,0.051588,0.0,0.0,0.0,0.0,0.0,0.120410,0.025820,0.0,0.0,...,0.036503,0.0,0.0,0.044692,0.0,0.068254,0.025820,0.025820,0.0,0.0,0.0,0.025820,0.0,0.0,0.025820,0.0,0.077253,0.0,0.0,0.0,0.025820,0.0,0.036503,0.0,0.036503,0.085349,0.036503,0.051588,0.044692,0.044692,0.0,0.0,0.132995,0.341663,0.092722,0.051588,0.0,0.0,0.092837,0.344314
min,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
25%,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
50%,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
75%,0.00

# 5. Implement the Naive Bayes from scratch, fit the respective models to the training data.

In [45]:
class NaiveBayes():
  def __init__(self, y):
    self.num_classes = len(np.unique(y)) # Get number of classes (number of unique value in y) = 2 (spam=1 and ham=0)
    self.alpha = 1

  def fit(self, X, y):
    c = y.value_counts() # Counting the total number of each class
    self.total_ham = c[0]
    self.total_spam = c[1]
    self.word_count_per_class = pd.concat([X, y], axis=1).groupby([y.name]).sum() # Creating a dataframe in which the total number of each word per class is stored

  def predict(self, X):
    probs = np.zeros((X.shape[0], 2)) # Array storing the results

    for index, row in X.iterrows(): # Looping through each mail
      score_ham = self.total_ham / (self.total_ham + self.total_spam) # Base ham score for a new mail is the probability of having a ham mail
      score_spam = self.total_spam / (self.total_ham + self.total_spam) # Base spam score for a new mail is the probability of having a spam mail

      for word in row[row != 0].iteritems(): # Looping through each word of a mail (only word that have a count that is not null)
        score_ham *= (self.alpha + self.word_count_per_class.iloc[0][word[0]])/(self.total_ham+self.alpha*self.total_ham) # Multiplicating the ham score by the probability to find the current word in a ham mail
        score_spam *= (self.alpha + self.word_count_per_class.iloc[1][word[0]])/(self.total_ham+self.alpha*self.total_ham) # Multiplicating the spam score by the probability to find the current word in a spam mail
      
      # Setting the scores for the mail at index
      probs[index][0] = score_ham
      probs[index][1] = score_spam

    # returning only the highest probability for each mail
    return np.argmax(probs, 1)





In [46]:
NB = NaiveBayes()

In [47]:
NB.fit(training_data_frame.drop(columns=["value => y_train"]), training_data_frame["value => y_train"])

In [48]:
y_pred = NB.predict(training_data_frame.drop(columns=["value => y_train"]))

In [49]:
sum(y_pred == training_data_frame["value => y_train"])/len(y_pred)


0.9031428571428571

# 6. Make predictions for the test data.

In [50]:
y_pred = NB.predict(test_data_frame.drop(columns=["value => y_test"]))

In [51]:
sum(y_pred == test_data_frame["value => y_test"])/len(y_pred)


0.904

# 7. Measure the spam-filtering performance for each approach through the confusion matrix.

In [52]:
y_true = test_data_frame["value => y_test"]

# Like we are searching for spams we do consider spam as positive
true_ham = 0 # true negatives
false_ham = 0 # false negatives
true_spam = 0 # true positive
false_spam = 0 # false positive

for i in range(len(y_true)):
  if y_true[i] == y_pred[i]:
    if y_true[i] == 0:
      true_ham += 1
    else:
      true_spam += 1
  else:
    if y_pred[i] == 0:
      false_ham += 1
    else:
      false_spam += 1

print("true spam : ",true_spam, " |   false spam : ",false_spam )
print("----------------------------------------")
print("false ham : ",false_ham, "  |   True ham   : ",true_ham )

true spam :  63  |   false spam :  1
----------------------------------------
false ham :  143   |   True ham   :  1293


# 8. Discuss your results.

The algorithm succeed in classifying a lot of ham into ham and a lot of spam into spam but didn't filter enough the spam as we have 143 spam that have been misclassified as ham.
On the other hand, the algorithm was good at detecting ham as he misclassified only one ham as spam.
We can tell that our algorithm is a bit underfitting.
Furthermore, as we have almost the same accuracy between the training and the test, we can tell that our alogirthm has a low bias.